# 06 · Regression analysis (prospective: logistic & survival)

> **Purpose**: analyse incident outcomes using logistic regression (risk over follow-up) and time-to-event models (Cox proportional hazards).

> **Learning objectives**
- Construct follow-up time and censoring from baseline and event dates.
- Fit incident-outcome **logistic regression** (didactic) and interpret ORs.
- Fit a **Cox PH** model with `statsmodels` (PHReg) and interpret hazard ratios.
- Run light diagnostics: PH reasonableness, non-linearity, sensitivity to adjustment sets.

---

In [ ]:
# Bootstrap: ensure repo root on path, then import init
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
from scripts.bootstrap import init
df, ctx = init()
df.head(2)

## 1) Build a survival dataset
We need **time** from baseline to event (or censor) and an **event** indicator (1=event, 0=censored). If there’s no explicit study end date, we assume **administrative censoring at 10 years** (teaching default). Edit the outcome/exposure below to change question.

_Default example_: `salt_g_d → CVD_incident` (confounders: age, sex, SES, IMD, smoking, BMI).

In [ ]:
import pandas as pd, numpy as np

# === EDIT THESE FOR YOUR QUESTION ===
OUTCOME_FLAG = 'CVD_incident'       # or 'Cancer_incident'
OUTCOME_DATE = 'CVD_date'           # or 'Cancer_date'
EXPOSURE     = 'salt_g_d'           # e.g., 'red_meat_g_d'
ADJUST = ['age','sex','BMI','SES_class','IMD_quintile','smoking_status']
# ===================================

use = ['id','baseline_date', OUTCOME_FLAG, OUTCOME_DATE, EXPOSURE] + ADJUST
surv = df[use].copy()

# Parse dates (robust to string/NA)
for c in ['baseline_date', OUTCOME_DATE]:
    if c in surv:
        surv[c] = pd.to_datetime(surv[c], errors='coerce')

# Administrative censor if no event: 10 years after baseline (teaching default)
admin_censor_days = 3650
surv['censor_date'] = surv['baseline_date'] + pd.to_timedelta(admin_censor_days, unit='D')
surv['event'] = surv[OUTCOME_FLAG].fillna(0).astype(int)
surv['event_date'] = np.where(surv['event']==1, surv[OUTCOME_DATE], pd.NaT)
surv['time_end'] = np.where(surv['event']==1, surv['event_date'], surv['censor_date'])
surv['time_days'] = (pd.to_datetime(surv['time_end']) - pd.to_datetime(surv['baseline_date'])).dt.days
surv['time_years'] = surv['time_days'] / 365.25

# Drop impossible/negative times
surv = surv[(surv['time_years'] > 0) & ~surv['time_years'].isna()].copy()
surv[['time_years','event']].describe().T

_Sanity_: median follow-up should be several years; event proportion matches cohort design (roughly 10–12% by construction).

## 2) Incident logistic regression (teaching)
Treat the **incident flag** as a binary outcome and model it with logistic regression. This ignores varying follow-up time but is useful as a didactic baseline; **Cox** below is preferred for time-to-event with censoring.

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices

work = surv.dropna(subset=[OUTCOME_FLAG, EXPOSURE]).copy()
work['exp_log1p'] = np.log1p(work[EXPOSURE])  # often helpful

def cat_term(df_, v):
    return f"C({v})" if (v in df_.columns and (df_[v].dtype=='object' or str(df_[v].dtype).startswith('category'))) else v

rhs_u = 'exp_log1p'
y_u, X_u = dmatrices(f'{OUTCOME_FLAG} ~ {rhs_u}', data=work, return_type='dataframe')
fit_log_u = sm.Logit(y_u, X_u).fit(disp=False)

rhs_a = ' + '.join(['exp_log1p'] + [cat_term(work, v) for v in ADJUST])
y_a, X_a = dmatrices(f'{OUTCOME_FLAG} ~ ' + rhs_a, data=work, return_type='dataframe')
fit_log_a = sm.Logit(y_a, X_a).fit(disp=False)

def tidy_or(f):
    OR = np.exp(f.params).rename('OR')
    CI = np.exp(f.conf_int()).rename(columns={0:'2.5%',1:'97.5%'})
    return pd.concat([OR,CI], axis=1).round(3)

tidy_or(fit_log_u).filter(like='exp_log1p', axis=0), tidy_or(fit_log_a).filter(like='exp_log1p', axis=0)

**Interpretation**: OR for `exp_log1p` approximates the multiplicative change in **odds** of having an event during follow-up per 1-unit increase in log(1+exposure), adjusted for covariates. Use hazard ratios from Cox below when time matters (it usually does).

## 3) Cox proportional hazards (PHReg, statsmodels)
We model **time_years** with censoring using the Cox PH model as implemented in `statsmodels.duration.hazard_regression.PHReg`. We’ll include the exposure (log1p) and the same confounders. (No extra dependencies.)

In [ ]:
from statsmodels.duration.hazard_regression import PHReg

# Build design: encode categoricals as dummies (no intercept; PHReg handles baseline via partial likelihood)
Z = work[['exp_log1p'] + ADJUST].copy()
Z = pd.get_dummies(Z, drop_first=True)

endog = work['time_years']
status = work['event']

cox = PHReg(endog, Z, status=status)
res_cox = cox.fit(disp=False)
res_cox.summary()  # shows log(HR) coefficients and SEs

**Extract hazard ratios (HR)** for readability:

In [ ]:
params = res_cox.params
conf   = res_cox.conf_int()
HR = np.exp(params).rename('HR')
HR_CI = np.exp(conf)
hr_tab = pd.concat([HR, HR_CI], axis=1).rename(columns={0:'2.5%',1:'97.5%'}).round(3)
hr_tab.filter(like='exp_log1p', axis=0)

**Interpretation**: HR > 1 for `exp_log1p` suggests higher hazard (instantaneous risk) with higher exposure, conditional on the covariates and the PH assumption.

### Non-linearity / scaling
As in cross-sectional models, it’s reasonable to consider a log transform (`log1p`) for skewed exposures and to test simple **curvature** (e.g., quadratic term) if diagnostics suggest it. Keep models interpretable; justify with EDA and clinical sense.

## 4) Light PH diagnostics
Formal tests are involved; here’s a **quick, pragmatic check**: interact key covariates with log(time) and test if interactions are ~0 (violations if large & significant). This is a *heuristic* rather than a full Schoenfeld analysis.

In [ ]:
work_diag = work.copy()
work_diag['log_t'] = np.log(work_diag['time_years'])
Z0 = pd.get_dummies(work_diag[['exp_log1p'] + ADJUST], drop_first=True)
Zt = Z0.mul(work_diag['log_t'], axis=0)
Z_ph = pd.concat([Z0, Zt.add_suffix(':log_t')], axis=1)

cox_ph = PHReg(work_diag['time_years'], Z_ph, status=work_diag['event'])
res_ph = cox_ph.fit(disp=False)
# Pull interaction rows only
ph_terms = res_ph.params[res_ph.params.index.str.endswith(':log_t')]
ph_ci = res_ph.conf_int().loc[ph_terms.index]
ph_tab = pd.concat([ph_terms.rename('coef'), ph_ci], axis=1).rename(columns={0:'2.5%',1:'97.5%'}).round(3)
ph_tab.head(10)

_Heuristic_: large interactions (CI far from 0) suggest PH issues for those covariates. If present, consider stratification, time-varying effects, or restricting follow-up periods, and **report the limitation** transparently.

## 5) Sensitivity: alternative adjustment sets
Compare unadjusted, minimal sufficient set (from your **DAG**), and an over-adjusted set (including a potential **mediator**) to illustrate estimate movement. Keep your primary estimand clear (total vs direct effect).

In [ ]:
def cox_fit(exposure, adjust_vars):
    Z = pd.get_dummies(work[['exp_log1p'] + adjust_vars].rename(columns={exposure:'exp_log1p'}), drop_first=True)
    model = PHReg(work['time_years'], Z, status=work['event'])
    res = model.fit(disp=False)
    # return HR for the exposure term
    b = res.params['exp_log1p']
    lo, hi = res.conf_int().loc['exp_log1p']
    return float(np.exp(b)), float(np.exp(lo)), float(np.exp(hi))

sets = {
  'Unadjusted': [],
  'Minimal DAG set': ['age','sex','SES_class','IMD_quintile','smoking_status'],
  'Over-adjusted (adds BMI)': ['age','sex','SES_class','IMD_quintile','smoking_status','BMI']
}
rows = []
for name, s in sets.items():
    try:
        hr, lo, hi = cox_fit(EXPOSURE, s)
        rows.append({'Model': name, 'HR': round(hr,3), '2.5%': round(lo,3), '97.5%': round(hi,3)})
    except Exception as e:
        rows.append({'Model': name, 'HR': np.nan, '2.5%': np.nan, '97.5%': np.nan})
import pandas as pd
pd.DataFrame(rows)

## 6) # TODO — your practice
1. Swap to **`EXPOSURE='red_meat_g_d'`** and **`OUTCOME='Cancer_incident'`**; repeat the logistic and Cox analyses. Report OR and HR with 95% CI.
2. Test a non-linear exposure term (e.g., **quadratic** or log1p vs raw). Does fit or interpretation improve?
3. Rerun Cox with your **minimal sufficient adjustment set** from the DAG notebook. How does the exposure HR move vs unadjusted and over-adjusted?
4. Briefly comment on **PH reasonableness** using the log-time interactions table. Any large deviations?
5. (Optional) Export a tidy CSV with your final HR result for the assessment submission.

In [ ]:
# (5) Export final HR for the exposure (edit label as needed)
final = {'Exposure': EXPOSURE, 'Outcome': OUTCOME_FLAG, 'HR': hr_tab.loc['exp_log1p','HR'] if 'HR' in hr_tab else np.nan,
         'CI_low': hr_tab.loc['exp_log1p','2.5%'] if 'HR' in hr_tab else np.nan,
         'CI_high': hr_tab.loc['exp_log1p','97.5%'] if 'HR' in hr_tab else np.nan}
pd.DataFrame([final]).to_csv('submission/final_hr.csv', index=False)
print('Saved submission/final_hr.csv')

> ## Key takeaways
>
> - Construct time & censoring carefully; Cox PH is the standard for time-to-event with censoring.
> - Logistic on the incident flag is **didactic**; report ORs but prefer HRs when time varies.
> - Keep transformations and non-linearity **motivated** by EDA and diagnostics.
> - Adjustment sets should follow your **DAG**; show how estimates move under different sets and discuss why.
> - Document assumptions (PH, measurement error, residual confounding) and provide a brief sensitivity narrative.